<a href="https://colab.research.google.com/github/cchen744/CornYield_NN/blob/b1.2/LSTM_%2B_MLP_model_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

(Optional) Get datasets from Github

In [2]:
# Access github repo
REPO_URL = "https://github.com/cchen744/CornYield_NN.git"
REPO_NAME = "CornYield_NN" # This is the folder name that will be created

# 4. Clone the repository
# We use the token for secure, authenticated access
!git clone https://github.com/cchen744/CornYield_NN.git

# 5. Change the working directory into the cloned repository folder
import os
os.chdir(REPO_NAME)

# Verify the files are there (you should see your notebook and dataset files)
print(f"Current directory contents in /{REPO_NAME}:")
!ls -F

Cloning into 'CornYield_NN'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 58 (delta 27), reused 26 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 3.18 MiB | 3.45 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Current directory contents in /CornYield_NN:
counties.csv		     prism_weather_1995-2009.csv  weather_clean.csv
final_dataset.csv	     prism_weather_2010-2024.csv  wide_data.csv
get_data.ipynb		     prism_weather.csv		  yield_clean.csv
LSTM_+_MLP_model.ipynb	     RandomForest.ipynb		  yield.csv
nasa_weather.csv	     soil_clean.csv
prism_weather_1984-1994.csv  soil_data.csv


#Dataset preparation

We have 3 types of datasets:
1. yield data, which looks like: <br>
Year, County, Yield
2. monthly weather data, which looks like: <br>
County, Year, Month, solar_radiation, humidity, ...., vpd_max
3. soil dataset: <br>
County, bdod, cec, ....

In [3]:
# Load 3 datasets
yield_df_path = "/content/CornYield_NN/yield_clean.csv"
soil_df_path = "/content/CornYield_NN/soil_clean.csv"
weather_df_path = "/content/CornYield_NN/weather_clean.csv"
yield_df = pd.read_csv(yield_df_path)
soil_df = pd.read_csv(soil_df_path)
weather_df = pd.read_csv(weather_df_path)
print(yield_df.head())
print(soil_df.head())
print(weather_df.head())

   Year      County  Yield
0  2024       adams  120.1
1  2024  green lake  168.3
2  2024      juneau  141.0
3  2024   marquette  126.0
4  2024     portage  159.5
     County  bdod  cec  clay  nitrogen  phh2o  sand  silt  soc
0     adams   132  163   192       698     60   489   319  375
1   ashland   115  249   258       601     51   216   526  605
2    barron   133  150   185       516     55   317   498  677
3  bayfield   111  266   187       545     50   480   333  668
4     brown   134  289   308       781     66   328   364  429
  County  Year  Month  solar_radiation  humidity  wind_speed  wind_speed_max  \
0  adams  1984      4            16.85     75.80        4.40           12.36   
1  adams  1984      5            17.63     74.25        3.26            9.64   
2  adams  1984      6            20.75     78.96        3.42            9.15   
3  adams  1984      7            21.90     74.48        2.68            8.19   
4  adams  1984      8            18.52     68.76        2.31

We want the dataset put into LSTM has the format:<br>
(batch_size,num_months,num_features)

In [4]:
# prepare monthly sequences

# Select monthly columns for LSTM
monthly_features = weather_df.columns[3:]
# Sort and group monthly dataset by (County, Year)
grouped_weather = (weather_df
    .sort_values(["County", "Year", "Month"])
    .groupby(["County", "Year"])
)

# Build dictionary: key = (county, year), value = monthly sequence array
monthly_weather_dict = {}

for (county, year), g in grouped_weather:
    seq = g[monthly_features].values  # shape = (num_months, num_features)
    monthly_weather_dict[(county, year)] = seq

# Show an example
first_key = list(monthly_weather_dict.keys())[0]
print("Example key:", first_key)
print("Sequence shape:", monthly_weather_dict[first_key].shape)
print(monthly_weather_dict[first_key][:3])  # first 3 months


Example key: ('adams', np.int64(1984))
Sequence shape: (6, 11)
[[16.85 75.8   4.4  12.36  3.83 35.5  46.5  57.5  29.7   1.57 10.85]
 [17.63 74.25  3.26  9.64  2.27 41.4  53.7  65.9  40.2   1.12 13.4 ]
 [20.75 78.96  3.42  9.15  5.96 56.2  68.1  80.   57.    1.2  18.43]]


Train test split

In [5]:
# Original
"""
# Find all (county, year) pairs with yield + weather keys, store keys in a list
all_keys = list(monthly_weather_dict.keys())
# Perform train_test_split on key_list
train_keys, test_keys = train_test_split(all_keys, test_size=0.2, random_state=42)
"""

'\n# Find all (county, year) pairs with yield + weather keys, store keys in a list\nall_keys = list(monthly_weather_dict.keys())\n# Perform train_test_split on key_list\ntrain_keys, test_keys = train_test_split(all_keys, test_size=0.2, random_state=42)\n'

In [6]:
# UPDATE skip data that is not present in both yield and weather

# Get all (county, year) pairs from monthly_weather_dict
all_weather_keys = set(monthly_weather_dict.keys())

# Ensure 'Year' column in yield_df is np.int64 to match weather_dict keys
yield_df['Year'] = yield_df['Year'].astype(np.int64)

# Get all (county, year) pairs from yield_df with consistent Year type
all_yield_keys = set(tuple(row) for row in yield_df[['County', 'Year']].values)

# Find the intersection of keys (only keep pairs that have both weather and yield data)
all_keys = list(all_weather_keys.intersection(all_yield_keys))

# Perform train_test_split on key_list
train_keys, test_keys = train_test_split(all_keys, test_size=0.2, random_state=42)

In [12]:
# UPDATE fixed issue with incorrect datatypes and assignments
# Dataset preparations -- GitHub
# Added: dataset normalization
class NormalizedYieldDataset(Dataset):
    def __init__(self, keys, yield_df, soil_df, weather_dict,
                 weather_scaler=None, soil_scaler=None, yield_scaler=None,
                 fit_scalers=False):
        self.keys = keys
        self.yield_df = yield_df
        self.soil_df = soil_df
        self.weather_dict = weather_dict

        # initialize scalers
        if fit_scalers:
            # Fit weather scaler
            all_weather = np.vstack([weather_dict[k] for k in keys])
            self.weather_scaler = StandardScaler()
            self.weather_scaler.fit(all_weather)

            # Fit soil scaler
            soil_counties = [k[0] for k in keys]
            soil_data = soil_df[soil_df['County'].isin(soil_counties)].drop('County', axis=1).values
            self.soil_scaler = StandardScaler()
            self.soil_scaler.fit(soil_data)

            # Fit yield scaler
            yield_data = []
            for county, year in keys:
                y = yield_df[(yield_df["County"]==county) & (yield_df["Year"]==year)]["Yield"].values[0]
                yield_data.append(y)
            self.yield_scaler = StandardScaler()
            self.yield_scaler.fit(np.array(yield_data).reshape(-1, 1))
        else:
            self.weather_scaler = weather_scaler
            self.soil_scaler = soil_scaler
            self.yield_scaler = yield_scaler

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        county, year = self.keys[idx]

        # Get and normalize weather
        weather_seq = self.weather_dict[(county, year)]
        weather_seq = self.weather_scaler.transform(weather_seq)
        weather_seq = torch.tensor(weather_seq, dtype=torch.float32)

        # Get and normalize yield
        y = self.yield_df[
            (self.yield_df["County"]==county) &
            (self.yield_df["Year"]==year)]["Yield"].values[0]
        y_normalized = self.yield_scaler.transform([[y]])[0]

        # Get and normalize soil
        soil_row = self.soil_df[self.soil_df["County"] == county].iloc[0]
        soil_vec = soil_row.drop("County").values.astype(np.float32)
        soil_vec = self.soil_scaler.transform([soil_vec])[0]
        soil_vec = torch.tensor(soil_vec, dtype=torch.float32)

        return weather_seq, soil_vec, torch.tensor(y_normalized, dtype=torch.float32)


In [13]:
# Create DataLoader
batch_size = 32
# create datasets
train_dataset = NormalizedYieldDataset(
    train_keys, yield_df, soil_df, monthly_weather_dict,
    fit_scalers=True
)

test_dataset = NormalizedYieldDataset(
    test_keys, yield_df, soil_df, monthly_weather_dict,
    weather_scaler=train_dataset.weather_scaler,
    soil_scaler=train_dataset.soil_scaler,
    yield_scaler=train_dataset.yield_scaler,
    fit_scalers=False
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
# Model
# Model assumes that there is at least 1 dimension in monthly, and static
class YieldLSTMMLPConnected(nn.Module):
    def __init__(self,
                    monthly_dim=11,     # Avg's by month (seq features, should be 11 features from weather_clean.csv)
                    monthly_layers=1,
                    monthly_hidden=64,
                    static_dim=8,    # number of static features (should be 8 features from soil data)
                    static_hidden=32,
                    head_hidden=64,
                    output_dim=1,
                    dropout=0.1
                    ):
        super().__init__()

        # Monthly branch LSTM
        self.lstm = nn.LSTM(
            input_size=monthly_dim,
            hidden_size=monthly_hidden,
            num_layers=monthly_layers,
            batch_first=True,
            bidirectional=False
        )
        self.monthly_proj = nn.Sequential(
            nn.Linear(monthly_hidden, monthly_hidden),
            nn.ReLU(),
            nn.Dropout(dropout)
        )


        # Static branch MLP
        self.static_proj = nn.Sequential(
            nn.Linear(static_dim, static_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(static_hidden, static_hidden),
            nn.ReLU(),
        )

        # Combined head (output of combined branches)
        combined_dim = monthly_hidden + static_hidden
        self.head = nn.Sequential(
            nn.Linear(combined_dim, head_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(head_hidden, head_hidden//2),
            nn.ReLU(),
            nn.Linear(head_hidden//2, output_dim)
        )

    def forward(self, weather_seq, static):
        feats = []

        # Monthly shape (batch, seq_len, monthly_dim)
        # LSTM: take last hidden state
        lstm_out, (h_n, c_n) = self.lstm(weather_seq)
        # h_n shape: (num_layers, batch, hidden)
        last_h = h_n[-1] # (batch, monthly_hidden)
        monthly_emb = self.monthly_proj(last_h)
        feats.append(monthly_emb)

        feats.append(self.static_proj(static))

        combined = torch.cat(feats, dim=1)
        out = self.head(combined)
        return out

In [15]:
# UPDATE removed yearly in for loops within train_loader
# Training
def train_model(
    model,
    train_loader,
    val_loader,
    num_epochs=50,
    lr=1e-3,
    weight_decay=1e-5,
    device="cuda" if torch.cuda.is_available() else "cpu",
    early_stop_patience=8
):

    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(1, num_epochs+1):
        # -------- TRAIN MODE --------
        model.train()
        train_losses = []

        for monthly, static, target in train_loader: # UPDATE removed yearly
            monthly = monthly.to(device)
            static = static.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            preds = model(monthly, static) # UPDATE removed yearly

            loss = criterion(preds, target)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        # -------- VAL MODE --------
        model.eval()
        val_losses = []

        with torch.no_grad():
            for monthly, static, target in val_loader: # UPDATE removed yearly
                monthly = monthly.to(device)
                static = static.to(device)
                target = target.to(device)

                preds = model(monthly, static) # UPDATE removed yearly
                loss = criterion(preds, target)
                val_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ---- EARLY STOP ----
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_yield_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= early_stop_patience:
                print("Early stopping triggered!")
                break

    print("Training completed. Best model saved as best_yield_model.pt")

In [16]:
# Run Testing
model = YieldLSTMMLPConnected()
train_model(model, train_loader, test_loader)

Epoch 001 | Train Loss: 0.8314 | Val Loss: 0.5432
Epoch 002 | Train Loss: 0.4605 | Val Loss: 0.4584
Epoch 003 | Train Loss: 0.3807 | Val Loss: 0.3993
Epoch 004 | Train Loss: 0.3464 | Val Loss: 0.3484
Epoch 005 | Train Loss: 0.3223 | Val Loss: 0.3441
Epoch 006 | Train Loss: 0.3005 | Val Loss: 0.3319
Epoch 007 | Train Loss: 0.2849 | Val Loss: 0.2964
Epoch 008 | Train Loss: 0.2729 | Val Loss: 0.3043
Epoch 009 | Train Loss: 0.2589 | Val Loss: 0.2660
Epoch 010 | Train Loss: 0.2369 | Val Loss: 0.2844
Epoch 011 | Train Loss: 0.2516 | Val Loss: 0.2780
Epoch 012 | Train Loss: 0.2375 | Val Loss: 0.2358
Epoch 013 | Train Loss: 0.2229 | Val Loss: 0.2245
Epoch 014 | Train Loss: 0.2053 | Val Loss: 0.2212
Epoch 015 | Train Loss: 0.1939 | Val Loss: 0.2098
Epoch 016 | Train Loss: 0.1863 | Val Loss: 0.2206
Epoch 017 | Train Loss: 0.1896 | Val Loss: 0.2013
Epoch 018 | Train Loss: 0.1760 | Val Loss: 0.2052
Epoch 019 | Train Loss: 0.1786 | Val Loss: 0.2091
Epoch 020 | Train Loss: 0.1744 | Val Loss: 0.2118


In [ ]:
# Load the best model
best_model = YieldLSTMMLPConnected()
best_model.load_state_dict(torch.load("best_yield_model.pt"))